In [13]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [21]:
class My_neural_network(nn.Module):
    def __init__(self):
        super(My_neural_network, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(1, 10, (3,3)),  # 10 imagenes de 26x26
            nn.MaxPool2d((3,3)), # 10 imagenes de 8x8

            nn.Conv2d(10, 28, (3,3)), # 28 imagenes de 6x6
            nn.MaxPool2d((3,3)), # 28 imagenes de 2x2

            nn.Flatten(),
            nn.Linear(28*2*2, 100),

            nn.ReLU(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = My_neural_network()
print(model)

My_neural_network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(10, 28, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=112, out_features=100, bias=True)
    (6): ReLU()
    (7): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [16]:
training_data = datasets.FashionMNIST(
  root="./tutorial/data",
  train=True,
  download=False,
  transform=ToTensor()
)

test_data = datasets.FashionMNIST(
  root="./tutorial/data",
  train=False,
  download=False,
  transform=ToTensor()
)


In [17]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
  size = len(dataloader.dataset)

  model.train()

  for batch, (X, y) in enumerate(dataloader):
    pred = model(X) #forward
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad() #reset all gradients

    if batch % 20 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  model.eval() # unnecessary in this situation but is a good practice
  #eval() === Test()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
def main(): 
  batch_size = 500
  learning_rate = 0.03
  epochs = 20

  train_dataloader = DataLoader(training_data, batch_size=batch_size)
  test_dataloader = DataLoader(test_data, batch_size=batch_size)

  model = My_neural_network()

  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n ---------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, batch_size)
    test_loop(test_dataloader, model, loss_fn)
  

main()


Epoch 1
 ---------------------------
loss: 2.302161  [  500/60000]
loss: 0.761299  [10500/60000]
loss: 0.507526  [20500/60000]
loss: 0.514392  [30500/60000]
loss: 0.521630  [40500/60000]
loss: 0.513792  [50500/60000]
Test Error: 
 Accuracy: 84.2%, Avg loss: 0.443029 

Epoch 2
 ---------------------------
loss: 0.361531  [  500/60000]
loss: 0.361718  [10500/60000]
loss: 0.348731  [20500/60000]
loss: 0.383789  [30500/60000]
loss: 0.409908  [40500/60000]
loss: 0.391930  [50500/60000]
Test Error: 
 Accuracy: 86.1%, Avg loss: 0.388414 

Epoch 3
 ---------------------------
loss: 0.290652  [  500/60000]
loss: 0.316350  [10500/60000]
loss: 0.305267  [20500/60000]
loss: 0.315029  [30500/60000]
loss: 0.380547  [40500/60000]
loss: 0.366865  [50500/60000]
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.365971 

Epoch 4
 ---------------------------
loss: 0.253859  [  500/60000]
loss: 0.329241  [10500/60000]
loss: 0.338074  [20500/60000]
loss: 0.289847  [30500/60000]
loss: 0.342164  [40500/60000]
loss: 